# BreweryPy
----

#### Note
* Instructions **

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import gmaps
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual
from scipy.stats import linregress
import scipy.stats as st
from pprint import pprint
from GeoLoc import GooglePlaces
# Google developer API key
from config import gkey
#print(gkey)



In [13]:
@interact
def show_articles_more_than(column='claps', x=5000):
    display(HTML(f'<h2>Showing articles with more than {x} {column}<h2>'))
    display(df.loc[df[column] > x, ['title', 'published_date', 'read_time', 'tags', 'views', 'reads']])

interactive(children=(Text(value='claps', description='column'), IntSlider(value=5000, description='x', max=15…

In [14]:
name_list = []
address_list = []
rating_list = []
lat_list = []
lng_list = []

if __name__ == '__main__':
    api = GooglePlaces(gkey)
    places = api.search_places_by_text("Brewery in Arizona, AZ", "restaurant,bar")
    #print(places)
    fields = ['name', 'formatted_address', 'rating', 'review','lat','lng'] 
    index_counter = 0
    for place in places:
        try:            
            name_list.append(place["name"])
            address_list.append(place["formatted_address"])
            rating_list.append(place["rating"])
            lat_list.append(place["geometry"]["location"]["lat"])
            lng_list.append(place["geometry"]["location"]["lng"])

            index_counter = index_counter + 1    
            #print(f"===================PLACE : ===================")
            #print("Name:", name_list)
            #print("Address:", address_list)
            #print("Rating:", rating_list)
            #print("Lat:", lat_list)
            #print("Lat:", lng_list)
            
        except(KeyError, IndexError):
            print("Brewery not found. Skipping...")
        
        

### Convert Brewery Raw Data to DataFrame
* Export the brewery data into a .csv.
* Display the DataFrame

In [15]:
# Create a panda data frame using data retrieved
brewery_df = pd.DataFrame({ 
                "Name" : name_list,
                "Address" : address_list,
                "Rating" : rating_list,
                "Lat" : lat_list,
                "Lng" : lng_list
       
})

In [16]:
#  Count data
brewery_df.count()

Name       0
Address    0
Rating     0
Lat        0
Lng        0
dtype: int64

In [17]:
# Display DataFrame
brewery_df.head()
#brewery_df.to_csv("DataFile/brewery.csv", index = False)

,Name,Address,Rating,Lat,Lng


In [18]:
# dropping null value columns to avoid errors 
brewery_df.dropna(inplace = True) 

# new data frame with split address columns 
new = brewery_df["Address"].str.split(",", n = 3, expand = True) 
  
# making separate Address column from new data frame 
brewery_df["Address"]= new[0] 
  
# making separate city column from new data frame 
brewery_df["City"]= new[1] 

# making separate State & Zipcode column from new data frame 
#brewery_df["State-Zip"]= new[2] 
brewery_df[['State','Zipcode']] = new[2].str.split(expand=True) 

# making separate Country column from new data frame 
brewery_df["Country"]= new[3] 
  
#df display 
brewery_df.head()

AttributeError: Can only use .str accessor with string values!

In [ ]:
# Export the Brewery Data into a csv
brewery_df.to_csv("DataFile/brewery.csv", index = False)

In [ ]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
locations = brewery_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = brewery_df["Rating"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>Address</dt><dd>{Address}</dd><dd>{City}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
bar_info = [info_box_template.format(**row) for index, row in brewery_df.iterrows()]
locations = brewery_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = bar_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig